In [6]:
%load_ext memory_profiler

import xarray, rioxarray
import rasterstats
import pandas, geopandas
import fsspec

In [2]:
mosaic_path = "../../data/ghs_composite_s2/GHS-composite-S2.vrt"
tst_path = "../../data/tessellation/tess_0.pq"

In [3]:
r = xarray.open_rasterio(mosaic_path,
                         chunks={"x": 1280,
                                 "y": 1280
                                }
                        )
ndvi = (r.sel(band=4) - r.sel(band=1)) / \
       (r.sel(band=4) + r.sel(band=1))
ndvi.rio.set_crs(r.rio.crs, inplace=True)

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


,Array,Chunk
Bytes,177.86 GB,13.11 MB
Shape,"(182437, 121865)","(1280, 1280)"
Count,82369 Tasks,13728 Chunks
Type,float64,numpy.ndarray


In [4]:
def ndvi_for_chunk(chunk_path, ndvi):
    with fsspec.open(chunk_path) as p:
        chunk = geopandas.read_parquet(p,
                                       columns=["hindex", "tessellation"]
                                      )
    chunk_ndvi = ndvi.rio.clip_box(*chunk.total_bounds)
    stats = rasterstats.zonal_stats(chunk,
                                    chunk_ndvi.values,
                                    affine=chunk_ndvi.rio.transform()
                                   )
    return pandas.DataFrame(stats)

In [7]:
%%memit 
cs = ndvi_for_chunk("../../data/tessellation/tess_0.pq", ndvi)

/opt/conda/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.7/site-packages/rasterstats/io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


peak memory: 2112.06 MiB, increment: 1879.55 MiB


In [8]:
%%memit 
cs = ndvi_for_chunk("../../data/tessellation/tess_83.pq", ndvi)

/opt/conda/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/conda/lib/python3.7/site-packages/rasterstats/io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


peak memory: 8627.19 MiB, increment: 7613.64 MiB


To do:

- Read window w/ rasterio and calculate ndvi inside the pod's task
- Deal with no data cells
- Parallelise in Dask